In [1]:
import os, json, pprint, elasticsearch, time, pickle

In [2]:
# 1 entity label-uri mapping
name_to_uri = json.load(open('/home/rovera/cm/scripts/cooccurrence/output/name_to_uri.json', 'r', encoding="utf-8"))
uri_to_name = json.load(open('/home/rovera/cm/scripts/cooccurrence/output/uri_to_name.json', 'r', encoding="utf-8"))
# 2 entity pages inverted index
ep_inv_index = pickle.load(open('/home/rovera/jl/scripts/entity_pages/ep/runs/run_0/ep_inv_index.pickle', 'rb'))
# 3 occurrence by journal
occurrence_by_journal = pickle.load(open('/home/rovera/cm/scripts/cooccurrence/output/occurrence_by_journal.pickle', 'rb'))
# 4 related entities
entity_correlation = pickle.load(open('/home/rovera/cm/scripts/cooccurrence/output/entity_correlation.pickle', 'rb'))
# 5 data types 
classified_entities = json.load(open('/home/rovera/cm/scripts/jl_linking/classified_entities.json', 'r', encoding="utf-8"))

In [3]:
# utilities
id_to_journal = json.load(open('/home/rovera/cm/scripts/cooccurrence/input/id_to_journal.json', 'r', encoding="utf-8"))

In [4]:
for name, uri in name_to_uri.items():
    print(name, uri)
    print(ep_inv_index[uri])
    print(occurrence_by_journal[uri])
    print(entity_correlation[uri])
    break

Franz Mehring http://data.judaicalink.org/data/dbpedia/Franz_Mehring
http://data.judaicalink.org/data/ep/1413226
[('3315709', 3), ('4886575', 3), ('2432763', 3), ('2316602', 3), ('2979076', 3), ('2827798', 2), ('2277379', 2), ('3062972', 2), ('2710055', 1), ('7506414', 1), ('2912821', 1), ('5367336', 1), ('2613366', 1), ('2583875', 1), ('2908689', 1), ('2641551', 1), ('9072829', 1), ('2692297', 1), ('2259275', 1)]
{'http://data.judaicalink.org/data/dbpedia/Hans_Blüher': 0.013986013986013986, 'http://data.judaicalink.org/data/dbpedia/Paul_Lindau': 0.010869565217391304, 'http://data.judaicalink.org/data/dbpedia/Maximilian_Harden': 0.006779661016949152, 'http://data.judaicalink.org/data/dbpedia/Alexander_Moszkowski': 0.023529411764705882, 'http://data.judaicalink.org/data/dbpedia/Friedrich_Engels': 0.017241379310344827, 'http://data.judaicalink.org/data/dbpedia/Anton_Pawlowitsch_Tschechow': 0.019417475728155338, 'http://data.judaicalink.org/data/dbpedia/Lazare_Sainéan': 0.0526315789473684

In [5]:
data = []
i = 0
for name, uri in name_to_uri.items():
    
    try:
        if i < 10:
            print(name)

        occ_by_journal = [(id_to_journal[el[0]], el[1]) for el in occurrence_by_journal[uri]]
        related_entities = sorted([(res_uri, uri_to_name[res_uri], score) for res_uri, score in entity_correlation[uri].items()], key=lambda x:x[2], reverse=True)
        label_name = name.replace(' ', '_')
        e_type = classified_entities[label_name]
        if len(e_type) == 0:
            e_type = "OTH"
        
        
        header = {'_id': str(i), '_index': 'cm_entities'}
        body = {'name': name, 
                'uri': uri,
                'e_type': e_type,
                'journal_occs': occ_by_journal,
                'related_entities': related_entities,
                'ep': ep_inv_index[uri]
               }


        data.append(json.dumps({"index": header}, ensure_ascii=False))
        data.append(json.dumps(body, ensure_ascii=False))
        i += 1
        
    except KeyError:
        pass
        #print(name, uri)
    

Franz Mehring
Hans Blüher
Wilhelm Börner (Philosoph)
Heinrich Mann
Aschersleben
Dresden
Głogów
Jelenia Góra
Wladimir Iljitsch Lenin
Pjotr Alexejewitsch Kropotkin


In [8]:
print(len(data))
print(data[1])

10890
{"name": "Franz Mehring", "uri": "http://data.judaicalink.org/data/dbpedia/Franz_Mehring", "e_type": "PER", "journal_occs": [["Die Welt : Zentralorgan der Zionistischen Bewegung", 3], ["Freies Blatt : Organ zur Abwehr des Antisemitismus", 3], ["Jüdisches Volksblatt", 3], ["Im deutschen Reich : Zeitschrift des Centralvereins Deutscher Staatsbürger Jüdischen Glaubens", 3], ["Wiener Morgenzeitung", 3], ["Freie Tribüne : Organ der jüdischen sozialdemokratischen Partei \"Poale Zion\"", 2], ["Central-Verein-Zeitung : Blätter für Deutschtum und Judentum ; C-V-Zeitung ; Organ des Central-Vereins Deutscher Staatsbürger Jüdischen Glaubens", 2], ["Die Wahrheit : unabhängige Zeitschrift für jüdische Interessen ; Deutschösterreichische Wochenschrift für jüdische Interessen", 2], ["Der neue Anfang : Zeitschr. der Jugend", 1], ["", 1], ["Menorah : jüdisches Familienblatt für Wissenschaft, Kunst und Literatur", 1], ["Das jüdische Literaturblatt", 1], ["Jüdisch-liberale Zeitung / Vereinigung für 

In [38]:
mappings = {
    'properties': {
        'name': {'type': 'text'},
        'uri': {'type': 'text'},
        'ep': {'type': 'text'},
        'e_type': {'type': 'text'}
        'journal_occs': {'type': 'text'},
        'related_entities': {'type': 'text'}
    }
}

In [39]:
es = elasticsearch.Elasticsearch()
ic = elasticsearch.client.IndicesClient(es)
if ic.exists('cm_entities'):
    ic.delete('cm_entities')
ic.create(index='cm_entities', body={'mappings': mappings})

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'cm_entities'}

In [42]:
es_index = es.bulk(data, request_timeout=200)

In [43]:
# test query
es = elasticsearch.Elasticsearch()
start = time.time()
res = es.search(index='cm_entities', body={'size': 10000, "query": {"match_phrase": {'name': 'Franz Mehring'}}})
end = time.time()
print("Search took", round(end-start, 2))

Search took 0.01


In [44]:
print("Total hits:", res['hits']['total']['value'])
for doc in res['hits']['hits']:
    print(doc['_source'])
    #print('Name:', doc['_source']['name'])
    #print('URI:', doc['_source']['uri'])
    #print('Entity Page:', doc['_source']['ep'])
    #for el in doc['_source']['journal_occs']:
    #    print(el)
    #print()

Total hits: 1
{'name': 'Franz Mehring', 'uri': 'http://data.judaicalink.org/data/dbpedia/Franz_Mehring', 'journal_occs': [['Die Welt : Zentralorgan der Zionistischen Bewegung', 3], ['Freies Blatt : Organ zur Abwehr des Antisemitismus', 3], ['Jüdisches Volksblatt', 3], ['Im deutschen Reich : Zeitschrift des Centralvereins Deutscher Staatsbürger Jüdischen Glaubens', 3], ['Wiener Morgenzeitung', 3], ['Freie Tribüne : Organ der jüdischen sozialdemokratischen Partei "Poale Zion"', 2], ['Central-Verein-Zeitung : Blätter für Deutschtum und Judentum ; C-V-Zeitung ; Organ des Central-Vereins Deutscher Staatsbürger Jüdischen Glaubens', 2], ['Die Wahrheit : unabhängige Zeitschrift für jüdische Interessen ; Deutschösterreichische Wochenschrift für jüdische Interessen', 2], ['Der neue Anfang : Zeitschr. der Jugend', 1], ['', 1], ['Menorah : jüdisches Familienblatt für Wissenschaft, Kunst und Literatur', 1], ['Das jüdische Literaturblatt', 1], ['Jüdisch-liberale Zeitung / Vereinigung für das Liberal